In [1]:
# Adapted from Lecture 8 Code
# The MLP that is trained on 3 cards and if the player wins or loses

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd

#Imports the google drive
from google.colab import drive
drive.mount('/content/drive')

#Changed to the correct directory inside the shared drive
import os
os.chdir("drive/Shared drives/COSC-480/COSC-480 Project/Sprint 1")

#Reads the correct CSV file inside the shared drive
df = pd.read_csv('blkjckhands.csv')

# Filter out rows where 'winloss' is 'Push'
df = df[df['winloss'] != 'Push']
df = df[df['card3'] == 0]

#Using a dictionary made adapting from the iris set program easier
madeUpData = {
    #Uses the pandas dataframe to assign the 3 columns as a nested list
    #card1, card2, dealcard1

    'data': df[['card1', 'card2', 'dealcard1']][0:10000].values.tolist(),

    #If we want to give the NN access to the dealers second card
    #'data': df[['card1', 'card2', 'dealcard1', 'dealcard2']][0:10000].values.tolist(),

    #If we want to test splitting
    #'data': df[['card1', 'dealcard1']][0:10000].values.tolist(),

    #Uses the pandas dataframe to assign the output index column
    #Replaces "Loss" with 0 and "Win" with 1 to actually use the classes
    'target': df['winloss'][0:10000].replace({'Loss': 0, 'Win': 1}).values.tolist()
}

X, y = madeUpData['data'], madeUpData['target'] ## Dataset Size X 3 ## Dataset Size X 1



                                                     ## 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train) # uses mean and standard deviation of the data to move the data
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.long) # Long used because other functions will be expecting it

class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

input_size = X_train.shape[1] # 3
hidden_size = 8 # just left this number for now
num_classes = 2 # win/loss
model = MLP(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()

#Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr = 0.001)

batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

#30 epochs was randomly assigned for now but seems to give similar results to 100
num_epochs = 30
for epoch in range(num_epochs):
  for batch_X, batch_y in train_loader: # Process each batch separately

    # Forward pass
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
  outputs = model(X_test)
  _, predicted = torch.max(outputs, 1)

accuracy = accuracy_score(y_test, predicted)
print(f'Accuracy: {accuracy * 100:.2f}%')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-3265769097.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'target': df['winloss'][0:10000].replace({'Loss': 0, 'Win': 1}).values.tolist()


Epoch [10/30], Loss: 0.4851
Epoch [20/30], Loss: 0.3602
Epoch [30/30], Loss: 0.6863
Accuracy: 71.25%


In [7]:
import torch.nn.functional as F

#  User input
card1 = int(input("What is your first card: "))
card2 = int(input("What is your second card: "))
dealerCard = int(input("What is the dealer's face up card: "))

user_input = torch.tensor([[card1, card2, dealerCard]], dtype=torch.float32)
user_input = scaler.transform(user_input)
user_input = torch.tensor(user_input, dtype=torch.float32)

with torch.no_grad():
    outputs = model(user_input)
    probs = F.softmax(outputs, dim=1).squeeze()

# Mapping
dealer_win_prob = probs[0].item() * 100
stand_prob = probs[1].item() * 100

#  Display nicely
print(f"{'Stand %:':15}{stand_prob:.1f}%")
print(f"{'Dealer Win %:':15}{dealer_win_prob:.1f}%")

#  still show final decision
if stand_prob > dealer_win_prob:
    print("\nYou are predicted to WIN if you stand!")
else:
    print("\nDealer is predicted to win.")

What is your first card: 10
What is your second card: 7
What is the dealer's face up card: 6
Stand %:       44.1%
Dealer Win %:  55.9%

Dealer is predicted to win.


In [ ]:
# Adapted from Lecture 8 Code
# My experiment to see if taking a hit is likely a win

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd

#Imports the google drive
from google.colab import drive
drive.mount('/content/drive')

#Changed to the correct directory inside the shared drive
import os
if os.path.exists("drive/Shared drives/COSC-480/COSC-480 Project/Sprint 1/blkjckhands.csv") == True:
  os.chdir("drive/Shared drives/COSC-480/COSC-480 Project/Sprint 1")

#Reads the correct CSV file inside the shared drive
df = pd.read_csv('blkjckhands.csv')

#Filter out rows where 'winloss' is 'Push'
df = df[df['winloss'] != 'Push']

#this mlp run is for only games that are longer than 2 player cards
df = df[df['card3'] > 0]

#replace the third card with 1 if it is actually used, 0 if it is not
df['card3'] = df.loc[df['card3'] > 0, 'card3'] = 1


#Using a dictionary made adapting from the iris set program easier
madeUpData = {
    #Uses the pandas dataframe to assign the 3 columns as a nested list
    #card1, card2, dealcard1

    'data': df[['card1', 'card2', 'card3', 'dealcard1']][0:10000].values.tolist(),

    #Uses the pandas dataframe to assign the output index column
    #Replaces "Loss" with 0 and "Win" with 1 to actually use the classes
    'target': df['winloss'][0:10000].replace({'Loss': 0, 'Win': 1}).values.tolist()
}

X, y = madeUpData['data'], madeUpData['target'] ## Dataset Size X 3 ## Dataset Size X 1



                                                     ## 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train) # uses mean and standard deviation of the data to move the data
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.long) # Long used because other functions will be expecting it

class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

input_size = X_train.shape[1] # 3
hidden_size = 8 # just left this number for now
num_classes = 2 # win/loss
model = MLP(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()

#Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr = 0.001)

batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

#30 epochs was randomly assigned for now but seems to give similar results to 100
num_epochs = 30
for epoch in range(num_epochs):
  for batch_X, batch_y in train_loader: # Process each batch separately

    # Forward pass
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
  outputs = model(X_test)
  _, predicted = torch.max(outputs, 1)

accuracy = accuracy_score(y_test, predicted)
print(f'Accuracy: {accuracy * 100:.2f}%')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1431630275.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'target': df['winloss'][0:10000].replace({'Loss': 0, 'Win': 1}).values.tolist()


Epoch [10/30], Loss: 0.6642
Epoch [20/30], Loss: 0.6721
Epoch [30/30], Loss: 0.6131
Accuracy: 59.95%


In [ ]:
#Same test as before but this should predict whether the inputed cards would win with or without a third card

import torch.nn.functional as F

#  User input
card1 = int(input("What is your first card: "))
card2 = int(input("What is your second card: "))
dealerCard = int(input("What is the dealer's face up card: "))

# Zero means no third card
user_input = torch.tensor([[card1, card2, dealerCard, 0]], dtype=torch.float32)
user_input = scaler.transform(user_input)
user_input = torch.tensor(user_input, dtype=torch.float32)

with torch.no_grad():
    outputs = model(user_input)
    probs = F.softmax(outputs, dim=1).squeeze()

# Mapping
dealer_win_prob = probs[0].item() * 100
stand_prob = probs[1].item() * 100


#hit_prob = (dealer_win_prob + stand_prob) / 2  # just example placeholder hit probability does not work

#  Display nicely
print(f"{'Stand %:':15}{stand_prob:.1f}%")
print(f"{'Dealer Win %:':15}{dealer_win_prob:.1f}%")

#  still show final decision
if stand_prob > dealer_win_prob:
    print("\nYou are predicted to WIN if you stand!")
else:
    print("\nDealer is predicted to win.")

# one means a third card is being used
user_input = torch.tensor([[card1, card2, dealerCard, 1]], dtype=torch.float32)
user_input = scaler.transform(user_input)
user_input = torch.tensor(user_input, dtype=torch.float32)

with torch.no_grad():
    outputs = model(user_input)
    probs = F.softmax(outputs, dim=1).squeeze()

dealer_win_prob = probs[0].item() * 100
hit_prob = probs[1].item() * 100

print(f"{'Hit %:':15}{hit_prob:.1f}%")

What is your first card: 6
What is your second card: 10
What is the dealer's face up card: 7
Stand %:       77.2%
Dealer Win %:  22.8%

You are predicted to WIN if you stand!
Hit %:         78.5%


In [ ]:
#Little test for pandas
import pandas as pd

df = pd.read_csv('blkjckhands.csv')

combined_row_data = df['winloss'].values.tolist()
print(combined_row_data[0:10])

combined_row_data = df['winloss'].replace({'Loss': 0, 'Win': 1}).values.tolist()
print(combined_row_data[0:10])

['Loss', 'Win', 'Loss', 'Loss', 'Loss', 'Loss', 'Win', 'Win', 'Win', 'Win']
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1]


In [ ]:
#More testing
import pandas as pd

df = pd.read_csv('blkjckhands.csv')

#'card1', 'card2', 'dealcard1'
data = df[['card1', 'card2', 'dealcard1']][0:1000].values.tolist()

print(data[0:20])

[[7, 10, 10], [10, 9, 10], [9, 8, 10], [2, 10, 10], [10, 2, 10], [6, 1, 10], [10, 4, 4], [3, 10, 4], [10, 2, 4], [9, 6, 4], [10, 4, 4], [3, 5, 4], [10, 2, 4], [9, 10, 4], [5, 1, 4], [10, 10, 4], [2, 9, 4], [9, 5, 4], [10, 4, 10], [2, 10, 10]]


In [ ]:
#More more testing
import pandas as pd

df['card3'] = df.loc[df['card3'] > 0, 'card3'] = 1
print(df['card3'][0:100])
#

0     1
1     1
2     1
3     1
4     1
     ..
95    1
96    1
97    1
98    1
99    1
Name: card3, Length: 100, dtype: int64
